# Very Random Coding Practices

## Design Questions

### Efficiently store and calculate dot products between embeddings
<p></p>
- [leetcode source](https://leetcode.com/discuss/interview-question/286446/facebook-interview-question-representation-and-dot-product-of-two-vectors)

    A: [1, 1, 4, 4, 4, 4, 7, 7, 7, 7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

    B: [2, 2, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
def compress_vector(A):
    if len(A) == 0:
        return []
    
    compress_A = [[A[0], 1]] # [value, count]
    for a in A[1:]:
        if a == compress_A[-1][0]:
            compress_A[-1][1] += 1
        else:
            compress_A.append([a, 1])
    return compress_A

def calculate_dot_product(A, B):
    
    compress_A = compress_vector(A)
    compress_B = compress_vector(B)
    
    i, j, dot_prod = 0, 0, 0
    len_A, len_B = len(compress_A), len(compress_B)
    while i < len_A and j < len_B:
        a, b = compress_A[i], compress_B[j]
        common = min(a[1], b[1])
        dot_prod += a[0] * b[0] * common
        
        a[1] -= common
        b[1] -= common
        
        if a[1] == 0:
            i += 1
        if b[1] == 0:
            j += 1
    return dot_prod


A = [1, 1, 4, 4, 4, 4, 7, 7, 7, 7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
B = [2, 2, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

assert calculate_dot_product(A, B) == sum([a * b for a, b in zip(A, B)])

### Build K-means from scratch using numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class kMeansNumpy:
    def __init__(self, k, dim=2):
        self.k = k
        self.centriod = np.zeros((dim, k))
    
    def fit(self, X, iteration=2):
        self.X = X
        self.cluster = np.zeros(len(self.X))
        self._initialize()
        fig, axs = plt.subplots(iteration+1,figsize=(5,15))
        self._plot(axs[0])
        
        plt_counter = 0
        while iteration >= 0:
            self._update_centriod()
            self._assign_cluster()
            self._plot(axs[plt_counter])
            plt_counter += 1
            iteration -= 1
        
    def _assign_cluster(self):
        for i, x in enumerate(self.X):
            distance = []
            for c in self.centriod:
                distance.append(self._get_distance(x, c))
            self.cluster[i] = np.argmin(distance)
        print('updated cluster is {}'.format(self.cluster))
        
    def _update_centriod(self):
        for c in range(self.k):
            X = self.X[self.cluster == c,]
            self.centriod[c] = np.mean(X,axis=0)
        print('updated cnetrioids are {}'.format(self.centriod))
        
    def _get_distance(self, x, y):
        return np.linalg.norm(x-y)
    
    def _initialize(self):
        m, _ = self.X.shape
        initial_k = np.random.choice(m, self.k, replace=False)
        self.centriod = self.X[initial_k,:]
        print('initialized cnetrioids are {}'.format(self.centriod))
        self._assign_cluster()
        
    def _plot(self, axis):
        axis.scatter(x=self.X[:, 0], y=self.X[:, 1], c=self.cluster)
        
X = np.array([[2, 3], [4, 5], [7, 7], [2, 7], [10, 2], [12, 2], [3, 2], [-3, 9], [0, 8], [1, 7], [4, 9], [-4, -3]])

classifier = kMeansNumpy(3)
classifier.fit(X)

### Build K-means from scratch using PySpark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

In [ ]:
class kMeansSpark:
    def __init__(self, k):
        self.k = k
    
    def _update_cluster(self):
        pass
    
    def _assign_centriod(self):
        pass
    
    def fit(self):
        pass

### Find k-closest points to origin
- [leetcode source](https://leetcode.com/problems/k-closest-points-to-origin/)
<p></p>
- three solutions depending on problem scale: is $N$ very big? is $K$ big or small?
<p></p>

    a) use a priority queue to maintain a list of k points, $O(Nlog(k))$;
    
    b) apply sorting, $O(Nlog(N))$;
    
    c) apply kd tree, $O(Nlog(N))$ for the first time to generate the tree, search, insert, remove all takes $O(log(N))$.
<p></p>
- a good practice on heapq

In [ ]:
import heapq

def find_k_closest_heap(points, k):
    candidates = []
    for x, y in points:
        dist = - x*x - y*y
        if len(candidates) == k:
            heapq.heappushpop(candidates, (dist, x, y))
        else:
            heapq.heappush(candidates, (dist, x, y))
    
    return [[x, y] for dist, x, y in candidates]


from scipy import spatial

def find_k_closest_kd_tree(points, k):
    kd_tree = spatial.KDTree(points)
    distance, idx = kd_tree.query(x=[0, 0], k=k, p=2)
    if k > 1:
        return [point for i, point in enumerate(points) if i in idx]
    else:
        return [points[idx]]
print(find_k_closest_heap([[3,3],[5,-1],[-2,4]], 2))
print(find_k_closest_kd_tree([[3,3],[5,-1],[-2,4]], 2))

## Maths Questions

### Inverse a matrix from scratch
1. loop over a focus_diagonal, e.g., 1,2,3;
2. for each focus_diagonal, loop over the other rows, deduct the current_row scalar from it;
3. list.copy() only copies this outter list, if it is a list of list, it doesn't copy the inner lists. copy.deepcopy() should be used instead.

In [ ]:
import numpy as np
import copy

def inverse_matrix(A):
    
    if not check_matrix_squareness(A):
        return []
    
    n = len(A)
    AM = copy.deepcopy(A)
    I = get_identity_matrix(n)
    row_index = list(range(n))
        
    for fd in range(n):
        try: 
            fd_scalar = 1.0 / AM[fd][fd]
        except ZeroDivisionError:
            return []
            
        for j in range(n):
            AM[fd][j] *= fd_scalar
            I[fd][j] *= fd_scalar
        
        for i in row_index[:fd]+row_index[fd+1:]:
            cr_scalar = AM[i][fd]
            for j in range(n):
                AM[i][j] = AM[i][j] - cr_scalar * AM[fd][j]
                I[i][j] = I[i][j] - cr_scalar * I[fd][j]

    return I

def get_identity_matrix(n):
    I = []
    for i in range(n):
        row = []
        for j in range(n):
            if i == j:
                row.append(1)
            else:
                row.append(0)
        I.append(row)
    return I


def check_matrix_squareness(A):
    if len(A) == 0 or len(A[0]) == 0 or len(A) != len(A[0]):
        return False
    else:
        return True
    
A = [[1,5,3], [4,5,6], [7,8,9]]

print(inverse_matrix(A))
print(np.linalg.inv(A))

### Check if a matrix is full rank

In [ ]:
A = [[1,2,3], [2,4,6], [7,8,9]]
if not inverse_matrix(A):
    print("A is not full rank")

### Solve a polynomial equation

In [ ]:
# use recursion

## Statistics Questions

### Write Batch Normalization from scratch

In [ ]:
import math

def batch_norm_dense(layer: list, epsilon: float, ita: float, beta: float):
    total = sum(layer)
    mu = total / len(layer)
    sigma = 0
    for ele in layer:
        sigma += (ele-mu)**2
    sigma /= len(layer)
    
    output = []
    for ele in layer:
        norm_ele = (ele - mu) / math.sqrt(sigma**2 + epsilon)
        output.append(ita * norm_ele + beta)
    return output

batch_norm_dense([1,2,3,4,5], 0.0001, 0.05, 0.3)

### Calculate precision, recall, AUC

In [ ]:
import matplotlib.pyplot as plt

def get_metrics(pred, true, threshold=0.5):
    pred_class = []
    for ele in pred:
        if ele >= threshold:
            pred_class.append(1)
        else:
            pred_class.append(0)
    
    TP, FN = 0, 0
    FP, TN = 0, 0
    for p, t in zip(pred_class, true):
        if t == 1:
            if p == 1:
                TP += 1
            else:
                FN += 1
        else:
            if p == 1:
                FP += 1
            else:
                TN += 1
    precision = TP / (TP + FP) if (TP+FP) else 1
    recall = TP / (TP + FN) if (TP+FN) else 1
    
    return precision, recall

def get_auc(pred, true):
    auc_pair = []
    plt.figure()
    for i in range(1, 11):
        auc_pair.append(get_metrics(pred, true, i/10))
    
    auc_pair.sort(key=lambda x: x[0])
    plt.step(auc_pair[0], auc_pair[1])

get_auc([0.7, 0.6, 0.3, 0.6, 0.1, 0.9], [1, 0, 0, 0, 1, 1])

## Algo Basics

### Sorting

In [ ]:
class Sort:
    def __init__(self, nums):
        self.nums = nums
    
    def do_quick_sort(self):
        self.quick_sort(0, len(self.nums)-1)
        return self.nums

    
    def partition(self, l, h):
        pivot = self.nums[h]
        pivot_i = l-1

        for i in range(l, h):
            if self.nums[i] < pivot:
                pivot_i += 1
                self.nums[pivot_i], self.nums[i] = self.nums[i], self.nums[pivot_i]
                

        self.nums[pivot_i+1], self.nums[h] = self.nums[h], self.nums[pivot_i+1]
        return pivot_i+1


    def quick_sort(self, l, h):
        if l < h:
            pivot_i = self.partition(l, h)
            self.quick_sort(l, pivot_i-1)
            self.quick_sort(pivot_i+1, h)



Sort([4,9,1,6,7,982,6,3,1]).do_quick_sort()

## Non-Leetcode Coding Questions

### Enumerate/Guess password combinations
Given the length of password n, and a list of unique digits where each must appear at least once. Guess the password

In [28]:
import random

def check(x):
    return x == "12312"

def guess_password_monte_carlo(n=5, digits=[1,2,3]):
    counter = 0
    cache = set()
    while True:
        counter += 1
        guess = ""
        for i in range(n):
            guess += str(random.choice(digits))
        if len(set(guess)) == len(digits) and guess not in cache:
            if check(guess):
                print(guess)
                return
            else:
                cache.add(guess)

def guess_password_backtrack(n=5, digits=[1,2,3]):
    
    def backtrack(path, unique_path):

        # early stopping
        if n - len(path) < len(digits) - len(unique_path):
            return
        
        # termination
        if len(path) == n:
            if len(unique_path) == len(digits):
                guess = "".join(path)
                if check(guess):
                    print(guess)
            return
        
        # enumerate
        for j in range(len(digits)):
            path.append(str(digits[j]))
            if str(digits[j]) in unique_path:
                unique_path[str(digits[j])] += 1
            else:
                unique_path[str(digits[j])] = 1
                
            # backtrack
            backtrack(path, unique_path)
            
            # revert
            path.pop()
            unique_path[str(digits[j])] -= 1
            if unique_path[str(digits[j])] == 0:
                del unique_path[str(digits[j])]

    backtrack([], {})
    

    
#         def backtrack(first = 1, curr = []):
#             # if the combination is done
#             if len(curr) == k:  
#                 output.append(curr[:])
#             for i in range(first, n + 1):
#                 # add i into the current combination
#                 curr.append(i)
#                 # use next integers to complete the combination
#                 backtrack(i + 1, curr)
#                 # backtrack
#                 curr.pop()

guess_password_monte_carlo()
guess_password_backtrack()


12312
12312


### Maze with Monsters 
A charcter walks from si, sj to ei, ej, return the shortest steps if possible.
A 2D array with:

0: empty space to walk through;

1: a wall that can not be passed;

6: a monster that reduces your health if you pass.

In [30]:
def get_shortest_steps(maze, h, si, sj, ei, ej):
    pass

### Maximum Path Sum from Leaf Nodes

In [7]:
class TreeNode:
    def __init__(self, val=0):
        self.val = val
        self.left = None
        self.right = None

n_n10 = TreeNode(-10)
n_n3 = TreeNode(-3)
n_20 = TreeNode(20)
n_15 = TreeNode(15)
n_n7 = TreeNode(-7)

n_n10.left = n_n3
n_n10.right = n_20
n_20.left = n_15
n_20.right = n_n7

def getMaxSumfromLeaf(root):
    if not root:
        return 0
    
    res = float("-Inf")
    
    def recursion(node):
        
        nonlocal res
        
        if not node:
            return 0
        
        # becasue it is leaf-leaf, we have to preserve all sum from child even they are negative
        left_sum = recursion(node.left)
        right_sum = recursion(node.right)
        
        # only update when current node is in the middle
        if node.left and node.right:
            res = max(res, left_sum+right_sum+node.val)
        
        return max(left_sum, right_sum) + node.val
    
    recursion(root)
    return res

print(getMaxSumfromLeaf(n_n10))

28
